In [1]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import csv
import time
import pandas as pd
import numpy as np
import pyodbc
import logging
import traceback, sys
import smtplib
from email.mime.text import MIMEText
import socket

In [2]:
# 日期計算
def init_time(data_back):
    """
    a_week: 一星期
    a_day: 一天
    end_time: 回推月份 - 1個月
    start_time: 回推月份
    """
    a_week = relativedelta(weeks = 1)
    a_day = relativedelta(days = 1)
    
    # 回推開始月份
    start_time = datetime.now() - data_back*relativedelta(months = 1)
    # 回推開始月份 - 1星期
    end_time = datetime.now() - data_back*relativedelta(months = 1) + relativedelta(weeks = 1)
    return start_time, end_time

In [8]:
def main_prog(start_time, end_time):
    # 起始值
    """
    access_token: 廣告金鑰
    ad_account_id: 廣告帳戶
    data_back: 往前爬多久的資料(幾個月)
    fields: FB_Ads行銷廣告資料
    count: 計算整體while迴圈跑幾次
    stop_flag: 當結束時間已經超過現在時間，使迴圈再跑一次，然後停止迴圈
    """
    access_token = 'EAAU5W2nIXuUBAIjps8JaNyhVzjh5mCywvWJDW6tleQXMGEl2xJMdlrfWDZB4wxZCRUU0Ed0FAKXT2acm31ZATt39m3u62ash9SP1La5BHgQOtEIBhCa5prRVDOuzZCPHAWeGPjBWCeZCegS3cmsu8ZBsfvURWFibQByAgGVqC53OHkxZA0pxixoCaXp1fKkLQgZD'
    ad_account_id = 'act_132799196821088'
    count = 0
    stop_flag = 0
    FacebookAdsApi.init(access_token = access_token)

    # Insight API中的fields資料(控制需要的欄位資料)
    fields = [
        'spend',
        'campaign_name',
        'adset_name',
        'campaign_id',
        'ad_name',
        'actions',
        'objective',
        'action_values',  
        'conversions',
        'conversion_values',
#         'frequency',
        'impressions',
#         'unique_ctr',
        'ctr',
        'cpm',
    ]

    # 起始資料放置位置
    Campaign_Name = list()
    Adset_Name = list()
    Ad_Name = list()
    Objective = list()
    Amount_Spent = list()
    Amount_Install = list()
    Amount_Purchase = list()
    Purchase_Conversion_Value = list()
    Amount_Subscribe = list()
    Subscribe_Conversion_Value = list()
    Reporting_Date = list()
    CPM = list()
    CTR = list()
    Impression = list()


    # 紀錄程式跑多久時間
    run_time = time.time()
   
    # 起始時間、結束時間(For Test)
#     start_time = datetime.now() - relativedelta(months = 5)
#     end_time = start_time + relativedelta(months = 1)
    
    # 起始時間、結束時間
    start_time, end_time = start_time, end_time
#     start_time, end_time = init_time(data_back = 13)

    # 現在時間
    now = datetime.now().date()

    # 比較時間前後順序
    while True:
        # 計算迴圈跑幾次
        count += 1
        # Insight API中的fields資料(控制日期條件、資料的層級)
        # start_time和end_time需要為string(%Y-%m-%d)
        params = {
            'time_range': {
#                         'since': str(start_time),\
#                         'until': str(end_time),
                        'since': start_time.strftime('%Y-%m-%d'),\
                        'until': end_time.strftime('%Y-%m-%d'), 
            }, 
            # level從campaign改成adset
            'level': {'ad'},
            'time_increment': '1',
            # 手動設定的歸因: 7天點擊後(和廣告預設值有落差)
            'action_attribution_windows': {'7d_click'},
        }

        # 廣告帳號獲取廣告Insights資訊
        acc_insights = AdAccount(ad_account_id).get_insights(
            fields = fields,
            params = params,
        )

        # 了解時間怎麼走
        print(start_time.strftime('%Y-%m-%d'))
        print(end_time.strftime('%Y-%m-%d'))
        
        # stop_flag = 0 代表end_time的時間已經超過現在時間(now_time)
        if stop_flag == 0:
        # 起始時間(start_time)和結束時間(end_time)，一次取一星期
            start_time = end_time + relativedelta(days = 1)
        else:
            start_time = end_time
        
        # 每次都將結束時間(end_time)，往後增加一星期
        end_time = end_time + relativedelta(weeks = 1)

        # 從廣告Inishgts資訊中，清洗、篩選資料
        for acc_insight in acc_insights:     
            campaign_name = ''
            adset_name = ''
            ad_name = ''
            amount_spent = ''
            objective = ''
            amount_subscribe = '' 
            subscribe_conversion_value = ''
            amount_install = '' 
            amount_purchase = ''    
            purchase_conversion_value = ''
            reporting_date = ''          
            cpm = ''
            ctr = ''
            impression = ''

            # insights/campaign_name得到 "廣告包名稱"
            if 'campaign_name' in acc_insight:
                campaign_name = acc_insight['campaign_name']
                if '⛔️股市爆料同學會' in campaign_name:
                    campaign_name = campaign_name.replace('⛔️', '')

            # insights/adset_name得到 "廣告set名稱"
            if 'adset_name' in acc_insight:
                adset_name = acc_insight['adset_name']
                
            # insights/adset_name得到 "廣告名稱"
            if 'ad_name' in acc_insight:
                ad_name = acc_insight['ad_name']
            
            # insights/cpm得到 "每一次廣告的曝光成本"
            if 'cpm' in acc_insight:
                cpm = acc_insight['cpm']
                
            # insights/ctr得到 "廣告點擊率"
            if 'ctr' in acc_insight:
                ctr = acc_insight['ctr']

            # insights/ctr得到 "廣告觸及人數"
            if 'impressions' in acc_insight:
                impression = acc_insight['impressions']
            
            # insights/spend得到 "總支出成本"
            if 'spend' in acc_insight:    
                amount_spent = acc_insight['spend']

            # insights/objective得到 "廣告目標"
            if 'objective' in acc_insight:
                objective = acc_insight['objective']

            # insights/date_stop得到 "報表終止日期"
            if 'date_stop' in acc_insight:
                reporting_date = datetime.strptime(acc_insight['date_stop'], '%Y-%m-%d').strftime('%Y%m%d')

            # insights/actions得到 "軟體下載數/課程購買數"
            if 'actions' in acc_insight:
                for action in acc_insight['actions']:
                    if action['action_type'] == 'mobile_app_install': 
                        amount_install = action['value']
                    if action['action_type'] == 'purchase':
                        amount_purchase = action['value']

            # insights/action_values得到 "購買轉換價值"
            if 'action_values' in acc_insight:
                for ac_values in acc_insight['action_values']:
                    if ac_values['action_type'] == 'omni_purchase':
                        purchase_conversion_value = ac_values['value']

            # insights/conversions得到 "訂閱數"
            if 'conversions' in acc_insight:
                for cv in acc_insight['conversions']:
                    if cv['action_type'] == 'subscribe_total':
                        try:
                            amount_subscribe = cv['7d_click']
                        except:
                            amount_subscribe = cv['value']

            # insights/conversion_values得到 "訂閱轉換價值"             
            if 'conversion_values' in acc_insight:
                for cv_values in acc_insight['conversion_values']:
                    if cv_values['action_type'] == 'subscribe_total':
                        try:
                            subscribe_conversion_value = cv_values['7d_click']
                        except:
                            subscribe_conversion_value = cv_values['value']

            # 將資料以List的形式儲存
            try:
                Campaign_Name.append(campaign_name)
                Adset_Name.append(adset_name)
                Ad_Name.append(ad_name)
                Objective.append(objective)
                Amount_Spent.append(amount_spent)
                Amount_Install.append(amount_install)
                Amount_Purchase.append(amount_purchase)
                Purchase_Conversion_Value.append(purchase_conversion_value)
                Amount_Subscribe.append(amount_subscribe)
                Subscribe_Conversion_Value.append(subscribe_conversion_value)
                Reporting_Date.append(reporting_date)
                CPM.append(cpm)
                CTR.append(ctr)
                Impression.append(impression)
            except UnicodeEncodeError as e:
                print(e)
            
            print(acc_insight['campaign_name'], '\n', acc_insight['adset_name'], '\n', acc_insight['ad_name'], '\n', acc_insight)
            
            # 如果結束時間大於現在時間，則立即停止迴圈
            if now < end_time.date():
                end_time = now
                stop_flag += 1        

        print('第{}次'.format(count)) 
        time.sleep(60)
        
        # 當成是執行4次時，暫停時間300秒
        if count%4 == 0:
            time.sleep(300)
        elif count == 2:
            break
        
        # 當開始時間和結束時間相等時，立即結束程式
        if start_time == end_time:
            break
        # 將資料以Dataaframe的方式儲存
        raw_data = {'Campaign_Name': Campaign_Name,
                 'Adset_Name': Adset_Name,
                 'Ad_Name': Ad_Name,
                 'Objective': Objective,
                 'Amount_Spent': Amount_Spent,
                 'Amount_Install': Amount_Install,
                 'Amount_Purchase': Amount_Purchase,
                 'Purchase_Conversion_Value': Purchase_Conversion_Value,
                 'Amount_Subscribe': Amount_Subscribe,
                 'Subscribe_Conversion_Value': Subscribe_Conversion_Value,
                 'CPM': CPM,
                 'CTR': CTR,
                 'Impression': Impression,
                 'Reporting_Date': Reporting_Date,}
        df = pd.DataFrame.from_dict(raw_data)

    # flag確立program完整結束
    print('--- %s seconds ---' % (time.time() - run_time))
    
    return df

In [9]:
start_time, end_time = init_time(13)
df = main_prog(start_time, end_time)

C:\ProgramData\Anaconda3\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of level might not be compatible.  Expect level_enum;  got <class 'set'>
  warnings.warn(message)
C:\ProgramData\Anaconda3\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of action_attribution_windows might not be compatible.  Expect list<action_attribution_windows_enum>;  got <class 'set'>
  warnings.warn(message)


2019-12-21
2019-12-28
期貨電子盤_下載_IOS(舊) 
 興趣期貨受眾_IOS 
 創意廣告 
 <AdsInsights> {
    "actions": [
        {
            "7d_click": "1",
            "action_type": "landing_page_view",
            "value": "1"
        },
        {
            "7d_click": "1",
            "action_type": "link_click",
            "value": "1"
        },
        {
            "7d_click": "1",
            "action_type": "post_reaction",
            "value": "1"
        },
        {
            "7d_click": "2",
            "action_type": "page_engagement",
            "value": "2"
        },
        {
            "7d_click": "2",
            "action_type": "post_engagement",
            "value": "2"
        }
    ],
    "ad_name": "\u5275\u610f\u5ee3\u544a",
    "adset_name": "\u8208\u8da3\u671f\u8ca8\u53d7\u773e_IOS",
    "campaign_id": "6098903020807",
    "campaign_name": "\u671f\u8ca8\u96fb\u5b50\u76e4_\u4e0b\u8f09_IOS(\u820a)",
    "cpm": "90.909091",
    "ctr": "9.090909",
    "date_start": "2019-12-21",
 

In [7]:
df

NameError: name 'df' is not defined